In [1]:
import inspect

from collections import OrderedDict

from typing import *
from mousse import validate

In [2]:
def cal_match_score(func: Callable, /, *args, **kwargs):
    signature = inspect.signature(func)
    parameters = signature.parameters
    
    parameters = OrderedDict(parameters.items())
    score = 0
    kwargs = kwargs.copy()
    
    has_var_args = False
    for param in parameters.values():
        if param.kind == param.VAR_POSITIONAL:
            has_var_args = True
            break
            
    if has_var_args:
        parameters.pop(param.name)
        
    has_var_kwargs = False
    for param in parameters.values():
        if param.kind == param.VAR_KEYWORD:
            has_var_kwargs = True
            break
            
    if has_var_kwargs:
        parameters.pop(param.name)

    positional_only_params = [
        param for param in parameters.values()
        if param.kind == param.POSITIONAL_ONLY
    ]

    if len(positional_only_params) > len(args):
        return -1
    
    for arg, param in zip(args, positional_only_params):
        parameters.pop(param.name)
    
        if param.annotation in (inspect._empty, Any):
            score += 1
            continue

        if validate(param.annotation, arg):
            score += 2
            continue
        
        return -1
        
    args = args[len(positional_only_params):]
    
    positional_or_keyword_params = [
        param for param in parameters.values()
        if param.kind == param.POSITIONAL_OR_KEYWORD
    ]
    
    if len(args) > len(positional_or_keyword_params):
        if not has_var_args:
            return -1
        score += 1
    
    for arg, param in zip(args, positional_or_keyword_params):
        parameters.pop(param.name)
        if param.annotation in (inspect._empty, Any):
            score += 1
            continue

        if validate(param.annotation, arg):
            score += 2
            continue

        return -1
        
    for key in kwargs:
        if key not in parameters:
            if has_var_kwargs:
                score += 1
                break

            return -1
    
    for key, val in kwargs.items():
        if key in parameters:
            param = parameters.pop(key)
            if param.annotation in (inspect._empty, Any):
                score += 1
                continue
                
            if validate(param.annotation, val):
                score += 2
                continue

            return -1
        
    for param in parameters.values():
        if param.default == inspect._empty:
            return -1
    
    return score

In [3]:
class Polymorphism:
    def __init__(self):
        self._funcs = []
        self._hashes = set()
        
    def __call__(self, *args, **kwargs) -> Any:
        best_matches = []
        best_score = -1
        
        for func in self._funcs:
            score = cal_match_score(func, *args, **kwargs)
            if score > best_score:
                best_score = score
                best_matches.clear()
                
            if score == best_score:
                best_matches.append(func)
                
        assert len(best_matches) <= 1, "Multiple matches found"
        assert len(best_matches) >= 0, "No match found"
        
        match = best_matches.pop()

        return match(*args, **kwargs)
    
    def __signature__(self):
        def dummy(*args, **kwargs) -> Any:
            pass
        return inspect.signature(dummy)
    
    def add(self, func: Callable):
        lines = inspect.getsource(func)
        lines = [
            line for line in inspect.getsource(func)
            if line.strip()
        ]
        func_hash = hash("".join(lines))
        
        if func_hash not in self._hashes:
            self._funcs.append(func)
            self._hashes.add(func_hash)
            
    def clear(self):
        self._funcs.clear()
        self._hashes.clear()
    
    
def polymorphism(func: Callable):
    _locals = inspect.currentframe().f_back.f_locals
    if func.__name__ not in _locals:
        _locals[func.__name__] = Polymorphism()

    poly = _locals[func.__name__]
    poly.add(func)
    
    return poly

In [4]:
@polymorphism
def foo():
    print("None")
    

@polymorphism
def foo(name: str):
    print(f"Name: {name}")
   

@polymorphism
def foo(age: int):
    print(f"Age: {age}")

In [5]:
foo.__signature__()

<Signature (*args, **kwargs) -> Any>

In [9]:
fooz.__code__.co_code

b't\x00d\x01|\x00\x9b\x00\x9d\x02\x83\x01\x01\x00d\x00S\x00'

In [16]:
inspect.signature(fooz).parameters["name"].default

inspect._empty

In [94]:
(1, 2).count()

AttributeError: 'tuple' object has no attribute 'clone'

In [7]:
foo